<a href="https://colab.research.google.com/github/Untick/IrisID_gr1/blob/Aigul-Jakypakunova-folder/%D0%97%D0%B5%D0%BD%D0%B8%D1%86%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# Сеть прямого распространения
from tensorflow.keras.models import Sequential 

# Базовые слои для свёрточных сетей
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

# Работа с изображениями
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

# Оптимизаторы
from tensorflow.keras.optimizers import Adam     

# Используем для to_categoricall
from tensorflow.keras import utils               

# Для отрисовки изображений
from tensorflow.keras.preprocessing import image 

from tensorflow.keras.callbacks import ModelCheckpoint

# для разделения выборок
from sklearn.model_selection import train_test_split

# Библиотека работы с массивами
import numpy as np

# Для отрисовки графиков
import matplotlib.pyplot as plt                  

# Для отрисовки изображений
from PIL import Image, ImageEnhance                       

# Для генерации случайных чисел 
import random                                    

# Для округления
import math                                      

# Для работы с файлами 
import os
import shutil
import zipfile     

# модуль для отсчета времени
import time                                   

# импорт tensorflow
import tensorflow as tf
# Для работы с файлами
import gc

import gdown

import numpy as np
import cv2
from scipy import ndimage
import math
import random

from google.colab.patches import cv2_imshow

# модуль для отсчета времени
import time 
from google.colab import drive
# Загрузка zip-архива с датасетом из облака на диск виртуальной машины colab
drive.mount('Eye_2')

Mounted at Eye_2


In [ ]:
data_dir = 'Eye_2'              # Путь к базе
os.listdir(data_dir)                                    # Список папок
batch_size= 48
image_size= (1920, 1080)

In [ ]:
def unwrap_circular_image(image):

    # Берем центр изображения
    height, width = image.shape[:2]
    center_x, center_y = width // 2, height // 2

    # Вычисляем максимальный радиус окружности
    max_radius = min(center_x, center_y)

    # Создаем новое изображение с теми же размерами, что и исходное изображение.
    unwrapped_image = np.zeros((2 * max_radius, 2 * max_radius, 3), dtype=np.uint8)

    # Итерируемся по каждому пикселю в новом изображении
    for y in range(unwrapped_image.shape[0]):
        for x in range(unwrapped_image.shape[1]):
            # Преобразуем координаты (x, y) в полярные
            r = np.sqrt((x - max_radius) ** 2 + (y - max_radius) ** 2)
            theta = np.arctan2(y - max_radius, x - max_radius)

            # Вычислим соответствующие (x, y) координаты в исходном изображении
            orig_x = int(center_x + r * np.cos(theta))
            orig_y = int(center_y + r * np.sin(theta))

            # Копируем значение пикселя из исходного изображения в новое изображение.
            if orig_x >= 0 and orig_x < width and orig_y >= 0 and orig_y < height:
                unwrapped_image[y, x] = image[orig_y, orig_x]


    return unwrapped_image

In [ ]:
def ring_to_rect(image):

  size = image.shape[0]  # assumes square image

  outer_radius = size // 2.05
  inner_radius_factor = 0.1  # 0.70 measured empirically from image

  # Разворачиваем кольцо
  warped = cv2.warpPolar(image, (size, int(size * math.pi)), (outer_radius, outer_radius), outer_radius, 0)
  # Поворачиваем на 15 градусов
  straightened = cv2.rotate(warped, cv2.ROTATE_15_COUNTERCLOCKWISE)
  # Обрезаем
  cropped = straightened[: int(straightened.shape[0] * (1 - inner_radius_factor)), :]

  return cropped
     


In [ ]:
def apply_gaussian_laplacian(image):

    ddepth = cv2.CV_16S
    kernel_size = 7
    
    image = cv2.GaussianBlur(image, (3, 3), 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.Laplacian(image, ddepth, ksize=kernel_size)
    image = cv2.convertScaleAbs(image)
    
    # Возвращаем изображение с примененными фильтром и оператором
    return image

In [ ]:
def gabor_features(image, ksize=31, sigma=5, theta=0, lambd=10, gamma=0.5):
    """
    Извлекает функции Габора из входного изображения в градациях серого с использованием фильтров Габора.
    :param image: Входное изображение в оттенках серого в виде массива NumPy.
    :param ksize: Размер ядра фильтра Габора.
    :param sigma: Стандартное отклонение конверта Гаусса.
    :param theta: Ориентация фильтра Габора в градусах.
    :param lambd: Длина волны синусоидального фактора.
    :param gamma: Пространственное соотношение сторон.
    :return: Обработанное изображение.
    """
    # Создаем ядро фильтра Габора
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)

    # Применияем фильтр Габора к входному изображению
    filtered = cv2.filter2D(image, cv2.CV_32F, kernel)

    # # Вычисляем среднее значение и стандартное отклонение отфильтрованного изображения
    mean, std = cv2.meanStdDev(filtered)

    # # Нормализуем отфильтрованное изображение
    normalized = (filtered - mean) / std

    # # Вычисляем величину и фазу сложного отклика вейвлета
    magnitude, phase = cv2.cartToPolar(np.real(normalized), np.imag(normalized))

    # # Вычисляем среднее значение и стандартное отклонение изображения магнитуды
    mean_mag, std_mag = cv2.meanStdDev(magnitude)

    # # Вычисляем среднее значение и стандартное отклонение фазового изображения
    mean_phase, std_phase = cv2.meanStdDev(phase)

    # # Объединяем вектор признаков
    feature_vector = np.concatenate((mean_mag, std_mag, mean_phase, std_phase))

    # # Преобразуем вектора признаков в изображение
    img_size = int(np.sqrt(len(feature_vector)))
    feature_image = feature_vector.reshape((img_size, img_size))

    return filtered


In [ ]:
# Получаем список всех файлов в папке и ее подпапках
import os

global Eye_2


file_list = []
for root, dirs, files in os.walk('Eye_2'):
    for file in files:
        file_list.append(os.path.join(root, file))

# Итерируемся по всем файлам и извлекаем числа из имен файлов
labels = []    
iris_images = []  

for file_path in file_list:
    # Получаем имя файла без расширения
    
    filename = os.path.splitext(os.path.basename(file_path))[0]
    filetext = os.path.splitext(os.path.basename(file_path))[1]
    # Получаем первые три числа из имени файла и убираем ведущие нули
    patient_num = int(''.join(filter(str.isdigit, filename))[:3])
    iris_images.append(filename+filetext)
    labels.append(patient_num)




In [ ]:

print('В массив собрано ', len(iris_images), ' фотографий')
print(iris_images)
print(labels)
     

В массив собрано  0  фотографий
[]
[]


In [ ]:
num_classes = 128

In [ ]:
# задаем желаемый размеры
scale_percent = 50                                            # создаем пустой список куда будем собирать массивы

iris_images

listID = []      # список путей до файлов изображений

for iris in iris_images:                          
  
    # открываем изображение и приводим его к заданному размеру
    img_iris = cv2.imread(output_path+iris)
    width = int(img_iris.shape[1] * scale_percent / 100)
    height = int(img_iris.shape[0] * scale_percent / 100)
    dim = (width, height)  

    img_iris = cv2.resize(img_iris, dim, interpolation = cv2.INTER_AREA)
    img_iris = unwrap_circular_image(img_iris)
    img_iris = ring_to_rect(img_iris)  
    img_iris = apply_gaussian_laplacian(img_iris)
    img_iris = gabor_features(img_iris)
    # cv2_imshow(img_iris)
    
    img_path = result_path + iris
    listID.append(img_path) # добавляем в общий список
    cv2.imwrite(img_path, img_iris)